In [ ]:
dir = '/content/drive/My Drive/DA_Project/YT'

In [ ]:
import numpy as np
import pandas as pd
import collections,os,json
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from IPython.core.display import HTML,display
from sklearn.ensemble import AdaBoostClassifier
import xgboost
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import VotingClassifier
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=UserWarning)
simplefilter(action='ignore', category=RuntimeWarning)
import pickle

In [ ]:
def set_css():
  return display(HTML("""<style>
  #output-body {
    display: flex;
    align-items: center;
    justify-content: center;
    background-color:white;
    color:black;
    }
  .output_text pre{
    margin-right : 70px;
    color: black;
    font-weight: bold;
    /* font-size: 10px; */
  }

}  </style>"""))

#Data Exploration

##Data Processing

In [ ]:
def process_date_attributes(data):
  data["publish_time"] = pd.to_datetime(data["publish_time"])
  data["trending_date"] = pd.to_datetime(data["trending_date"],format="%y.%d.%m")
  data["trend_year"]=data["trending_date"].apply(lambda time:time.year)
  data["trend_date"] = data["trending_date"].apply(lambda time: time.day)
  data["trend_month"]=data["trending_date"].apply(lambda time:time.month)
  data["publish_year"]=data["publish_time"].apply(lambda time:time.year)
  data["publish_month"]=data["publish_time"].apply(lambda time:time.month)
  data["publish_day"]=data["publish_time"].apply(lambda time:time.day)
  data["publish_weekday"]=data["publish_time"].apply(lambda time:time.dayofweek)
  data["publish_hour"]=data["publish_time"].apply(lambda time:time.hour)
  return data

In [ ]:
def match_categories(x,dictionary): #function to get the categories
    try:
        return dictionary[str(x)]
    except:
        return "Unknown"

##Data Cleaning

In [ ]:
def data_cleaning(data):
  if data['description'].isnull().sum()>0:
    data['description'] = data['description'].replace(np.nan,"") #Replacing the NaN values in the "Description" column with an empty string
  data = data.sort_values('video_id').drop_duplicates(subset=['trending_date', 'title'], keep='last')
  data = data.drop(columns=["publish_time","category_id"],axis=1)
  return data

##Loading the datasets

In [ ]:
def load_data(file_path): 
    ch_file=file_path
    Country_code = ch_file[0:2]
    path=os.path.join(dir,ch_file)
    input_df=pd.read_csv(path,encoding='utf-8')
    title_dict={}
    json_path=os.path.join(dir,(Country_code+str('_category_id.json')))
    json_input_df=pd.read_json(json_path)
    input_df['region']=Country_code
    for dict_a in json_input_df['items']:
        title_dict[dict_a['id']]=dict_a['snippet']['title'] 
    input_df = process_date_attributes(input_df) #Calling the function to process the date columns.
    input_df['category']=input_df['category_id'].apply(match_categories,args=(title_dict,))
    input_df = data_cleaning(input_df)
    return input_df

In [ ]:
set_css()
con_list = []
con_dict = {}
t_list = []
t_df = pd.DataFrame()
for dirname, _, filenames in os.walk(dir):
    print("................................")
    for paths in filenames:
        if paths.lower().endswith(".csv"):
          con_list.append(paths[0:2])
          x = load_data(paths)
          t_list.append(x)
          x = x.drop(columns=['video_id','region'],axis=1)
          globals()[str(paths[0:2])] = x
          con_dict[str(paths[0:2])]=x
          print("------Created",paths[0:2],"DataFrame------")
    print("Processed all available Datasets")
t_df = pd.concat(t_list,ignore_index=True)
tdf = t_df
tdf.name = "The 6 Countries"
print("................................")
print("tdf  is the cumulative DataFrame")

................................
------Created CA DataFrame------
------Created DE DataFrame------
------Created FR DataFrame------
------Created GB DataFrame------
------Created IN DataFrame------
Processed all available Datasets
................................
Processed all available Datasets
................................
tdf  is the cumulative DataFrame


In [ ]:
def get_unique_categ(data):
  cat_x = data['category'].unique()
  cat_y = list(cat_x)
  if "Unknown" in cat_y:
    cat_y.remove('Unknown')
  dictc = {}
  for i in range(0,len(cat_y)):
    dictc[i] = cat_y[i]
  return dictc,cat_x
# dict_cat,cat_x = get_unique_categ(tdf)

##Filling the Missing Categories using Word Vectorisers and Classification

In [ ]:
for i,j in con_dict.items():
  print(i,j[j.category=="Unknown"].value_counts().sum())

CA 74
DE 256
FR 114
GB 90
IN 103


In [ ]:
gnb_model = MultinomialNB()
rfc_model= RandomForestClassifier(n_estimators=20,verbose=False,n_jobs=-1)
dtc_model = DecisionTreeClassifier()
xgb_model=XGBClassifier()
knn_model = KNeighborsClassifier(n_neighbors=1)

In [ ]:
def create_model_data(data,nam):
  mdf = data[["title","category"]]
  mdf = mdf[mdf['category']!="Unknown"].reset_index()
  mdf['cid'] = 0
  dict_a,_ = get_unique_categ(data)
  for index,i in enumerate(mdf.category):
    for j in dict_a.keys():
      if dict_a[j]==i:
        mdf.at[index,'cid'] = j
  v = CountVectorizer()
  words = v.fit_transform(mdf['title'].values)
  cats = mdf.cid.values
  filename = nam + '_vector.pkl'
  pickle.dump(v, open(filename, 'wb'))
  return words,cats

In [ ]:
# def find_best_knn(X_train,y_train):
#   from sklearn.model_selection import GridSearchCV
#   knn2 = KNeighborsClassifier()
#   param_grid = {'n_neighbors': np.arange(1,8)}
#   knn_gscv = GridSearchCV(knn2, param_grid, cv=5)
#   knn_gscv.fit(X_train, y_train)
#   print("Optimal value for neighbors is->",knn_gscv.best_params_,"With accuracy->",knn_gscv.best_score_)
#   return list(knn_gscv.best_params_.values())[0]

In [ ]:
def create_ensemble(data,nam):
  print("-----------------Starting for",nam,"-----------------")
  X,y = create_model_data(data,nam)
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2)
  dtc = dtc_model.fit(X,y)
  gnb = gnb_model.fit(X,y)
  xgb = xgb_model.fit(X,y)
  rfc = rfc_model.fit(X,y)

  knn = knn_model.fit(X_train,y_train)
  print("----------------------------------------","Implementing Ensemble model has started","----------------------------------------")
  estimators=[('gnb_model', gnb), ('xgb_model', xgb), ('dtc_model',dtc), ('knn_model', knn), ('rfc_model', rfc)]
  # ens_name = "ensemble_" + str(nam)
  ens_name = VotingClassifier(estimators, voting='hard')
  ens_name.fit(X,y)
  acc_ens = ens_name.score(X_test, y_test)
  print("----------------------------------------","Implementing Ensemble model has finished","----------------------------------------")
  print("\nThe Ensemble Model for %s has achieved an accuracy of %.4f" %(nam,acc_ens))
  filename = nam + '_model.pkl'
  pickle.dump(ens_name, open(filename, 'wb'))
  return ens_name  

In [ ]:
def perform_ensemble():
  for i,j in con_dict.items():
    if j[j.category=="Unknown"].value_counts().sum() > 0:
      globals()[str("ensemble_") + str(i)]  = create_ensemble(j,i)
perform_ensemble()

-----------------Starting for CA -----------------
---------------------------------------- Implementing Ensemble model has started ----------------------------------------
---------------------------------------- Implementing Ensemble model has finished ----------------------------------------

The Ensemble Model for CA has achieved an accuracy of 0.9977
-----------------Starting for DE -----------------
---------------------------------------- Implementing Ensemble model has started ----------------------------------------
---------------------------------------- Implementing Ensemble model has finished ----------------------------------------

The Ensemble Model for DE has achieved an accuracy of 0.9978
-----------------Starting for FR -----------------
---------------------------------------- Implementing Ensemble model has started ----------------------------------------
---------------------------------------- Implementing Ensemble model has finished -----------------------------